In [14]:
## Importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import pickle

In [18]:
df = pd.read_csv("/Users/siddharth/Desktop/EMG_Classifier/emg/data/EMG-data.csv")

In [19]:
df.shape

(4237907, 11)

In [20]:
df.head()

,time,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8,class,label
0,1,0.00001,-0.00002,-0.00001,-0.00003,0.00000,-0.00001,0.00000,-0.00001,0,1
1,5,0.00001,-0.00002,-0.00001,-0.00003,0.00000,-0.00001,0.00000,-0.00001,0,1
2,6,-0.00001,0.00001,0.00002,0.00000,0.00001,-0.00002,-0.00001,0.00001,0,1
3,7,-0.00001,0.00001,0.00002,0.00000,0.00001,-0.00002,-0.00001,0.00001,0,1
4,8,-0.00001,0.00001,0.00002,0.00000,0.00001,-0.00002,-0.00001,0.00001,0,1


In [21]:
df.to_csv('/Users/siddharth/Desktop/EMG_Classifier/emg/transformed_data.csv')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4237907 entries, 0 to 4237906
Data columns (total 11 columns):
 #   Column    Dtype  
---  ------    -----  
 0   time      int64  
 1   channel1  float64
 2   channel2  float64
 3   channel3  float64
 4   channel4  float64
 5   channel5  float64
 6   channel6  float64
 7   channel7  float64
 8   channel8  float64
 9   class     int64  
 10  label     int64  
dtypes: float64(8), int64(3)
memory usage: 355.7 MB


In [23]:
df.drop(['label','time'], inplace=True, axis=1)

In [24]:
## Changing the datatype of class feature from object to numeric one
df['class'] = df['class'].apply(pd.to_numeric)

In [25]:
df.dropna(inplace=True)

In [27]:
## Reading the newly created data
df.to_csv('/Users/siddharth/Desktop/EMG_Classifier/emg/transformed_data.csv', index=False)

In [28]:
# df.head()

In [29]:
# df.shape

In [30]:
X, y = df.drop('class', axis=1), df['class']

In [31]:
X.head()

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
0,0.00001,-0.00002,-0.00001,-0.00003,0.00000,-0.00001,0.00000,-0.00001
1,0.00001,-0.00002,-0.00001,-0.00003,0.00000,-0.00001,0.00000,-0.00001
2,-0.00001,0.00001,0.00002,0.00000,0.00001,-0.00002,-0.00001,0.00001
3,-0.00001,0.00001,0.00002,0.00000,0.00001,-0.00002,-0.00001,0.00001
4,-0.00001,0.00001,0.00002,0.00000,0.00001,-0.00002,-0.00001,0.00001


In [32]:
X.shape

(4237907, 8)

In [33]:
y

0          0
1          0
2          0
3          0
4          0
          ..
4237902    0
4237903    0
4237904    0
4237905    0
4237906    0
Name: class, Length: 4237907, dtype: int64

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=34, stratify=y)

In [35]:
# Normalizing data
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train -= mean
X_train /= std

X_val -= mean
X_val /= std

In [36]:
## checking the number of rows and columns in the X_train
print(f"The number of rows in the train data are {X_train.shape[0]}.")
print(f"The number of columns in the train data are {X_train.shape[1]}.")

## checking the number of rows and columns in the X_test
print(f"The number of rows in the validation data are {X_val.shape[0]}.") 
print(f"The number of columns in the validation data are {X_val.shape[1]}.")

The number of rows in the train data are 3390325.
The number of columns in the train data are 8.
The number of rows in the validation data are 847582.
The number of columns in the validation data are 8.


In [37]:
## importing required tensorflow libraries
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow.keras.callbacks as tkc

In [38]:
# creating a keras model
input_tensor = layers.Input(shape=X_train.shape[1:])
x = layers.Dense(1024, activation='relu')(input_tensor)
y = layers.Dense(512, activation='relu')(x)
z = layers.Dense(256, activation='relu')(y)
z = layers.Dropout(0.2)(z)
z = layers.Dense(128, activation='relu')(z)
z = layers.Dense(64, activation='relu')(z)
z = layers.Dropout(0.2)(z)
z = layers.Dense(32, activation='relu')(z)
z = layers.Dense(128, activation='relu')(y) 
z = layers.Dense(64, activation='relu')(z)
z = layers.Dense(32, activation='relu')(z)
output_tensor = layers.Dense(8, activation='softmax')(z)

model = Model(input_tensor, output_tensor)

In [39]:
## compiling the model
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [40]:
## creating checkpoints -- ModelCheckpoint and EarlyStopping
checkpoint_cb = tkc.ModelCheckpoint("gesture_predictor.h5",save_weights_only=True, monitor='val_loss')
early_stopping_cb = tkc.EarlyStopping(patience=5,restore_best_weights=True)

In [41]:
## training the model using training and validation set
history = model.fit(X_train, y_train, epochs=200, batch_size=512, validation_data=(X_val, y_val),callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/200
6622/6622 [==============================] - 124s 19ms/step - loss: 0.9088 - accuracy: 0.6459 - val_loss: 0.8841 - val_accuracy: 0.6490
Epoch 2/200
6622/6622 [==============================] - 117s 18ms/step - loss: 0.8732 - accuracy: 0.6516 - val_loss: 0.8635 - val_accuracy: 0.6548
Epoch 3/200
6622/6622 [==============================] - 116s 18ms/step - loss: 0.8468 - accuracy: 0.6609 - val_loss: 0.8373 - val_accuracy: 0.6648
Epoch 4/200
6622/6622 [==============================] - 148s 22ms/step - loss: 0.8183 - accuracy: 0.6723 - val_loss: 0.8087 - val_accuracy: 0.6759
Epoch 5/200
 213/6622 [..............................] - ETA: 2:10 - loss: 0.8012 - accuracy: 0.6801

KeyboardInterrupt: 

In [ ]:
## visualizing the train and validation accuracy
sns.set_style('darkgrid')
plt.plot(history.history['accuracy'],color='black',label='training accuracy')
plt.plot(history.history['val_accuracy'],color='brown',label='validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
## visualizing the train and validation loss
plt.plot(history.history['loss'],color='black',label='training loss')
plt.plot(history.history['val_loss'],color='brown',label='validation loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()